## Transfer Learning VGG 16 and VGG 19 using Keras

Please download the dataset from the below url

https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

In [1]:
# import the libraries as shown below

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'dataset/training_set'
valid_path = 'dataset/test_set'


In [4]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [5]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [6]:
  # useful for getting number of output classes
folders = glob('dataset/training_set/*')

In [7]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [9]:

# view the structure of the model
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [11]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [13]:
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [14]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\ajayt\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5


InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,3] labels_size=[32,2]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at <ipython-input-14-6ec5a35725ca>:3) ]] [Op:__inference_train_function_1211]

Function call stack:
train_function


In [ ]:
from keras.models import load_model

model.save('cnnModelPneumoniaPrediction.h5')

In [ ]:
y_pred  = model.predict(test_set)

In [ ]:
y_pred


In [19]:
y_pred = np.argmax(y_pred, axis = 1)
y_pred

array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [21]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [28]:
new_model = load_model('cnnModel.h5')

In [97]:
test_image = image.load_img('dataset/single_prediction/images (5).jpg', target_size = (224,224))
test_image= image.img_to_array(test_image)
test_image

array([[[ 24.,  44.,  42.],
        [ 54.,  74.,  73.],
        [ 71.,  91.,  90.],
        ...,
        [195., 212., 219.],
        [195., 212., 219.],
        [194., 211., 218.]],

       [[ 20.,  40.,  39.],
        [ 51.,  71.,  70.],
        [ 68.,  88.,  89.],
        ...,
        [196., 213., 220.],
        [196., 213., 220.],
        [195., 212., 219.]],

       [[ 20.,  40.,  39.],
        [ 51.,  71.,  70.],
        [ 68.,  88.,  89.],
        ...,
        [196., 213., 220.],
        [196., 213., 220.],
        [195., 212., 219.]],

       ...,

       [[174., 148., 111.],
        [129., 107.,  70.],
        [105.,  85.,  50.],
        ...,
        [232., 225., 206.],
        [234., 226., 207.],
        [237., 226., 206.]],

       [[174., 148., 111.],
        [129., 107.,  70.],
        [105.,  85.,  50.],
        ...,
        [232., 225., 206.],
        [234., 226., 207.],
        [237., 226., 206.]],

       [[175., 149., 112.],
        [131., 109.,  72.],
        [106.,  

In [98]:
test_image.shape

(224, 224, 3)

In [99]:
#test_image = test_image/255

In [100]:
test_image = np.expand_dims(test_image, axis = 0)
img_data = preprocess_input(test_image)

In [101]:
img_data.shape

(1, 224, 224, 3)

In [102]:
new_model.predict(img_data)

array([[0., 1.]], dtype=float32)

In [103]:
a = np.argmax(new_model.predict(img_data), axis = 1)

In [104]:
if a == 1:
    print('dog')
else:
    print('cat')

dog


In [105]:
print(a)

[1]
